## MLflow의 저장 경로인 mlruns안의 파일 삭제.
1.27.0 버전의 mlflow에서는 mlflow ui에서 삭제한 run이 실제로 삭제되지 않는다. lifecycle이 delete로 표시가 될 뿐이므로 실제로 삭제를 하기 위해서는 일일히 삭제를 해야 한다.\
이 부분을 쉽게 처리하기위해 제작한 함수이다.\
meta.yaml 파일을 읽어 lifecycle을 체크한 후 delete인 run을 전부 삭제한다.

In [ ]:
import os
import yaml
import shutil

mlruns_path = "/home/work/model/multi_input_model/mlruns"
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as e:
            print(f'YAML 파일을 읽는 중 오류가 발생했습니다: {e}')

# mlruns에 있는 모든 run을 돌면서 delete상태인 run을 실제로 삭제함.
def convert_all():
    for experiment in os.listdir(mlruns_path):
        if not experiment.endswith('.yaml') and not experiment.startswith('.'):
            experiment_path = os.path.join(mlruns_path, experiment)
            convert_experiment(experiment_path)

# expriment안의 모든 runs
def convert_experiment(experiment_path):
    for run in os.listdir(experiment_path):
        if not run.endswith('.yaml'):
            run_path = os.path.join(experiment_path, run)
            try:
                convert_run(run_path)
            except:
                pass

# 하나의 run
def convert_run(run_path):
    yaml_path = os.path.join(run_path, "meta.yaml")
    
    try:
        data = read_yaml_file(yaml_path)
    except:
        pass
    
    try:
        lifecycle_stage = data['lifecycle_stage']
        if lifecycle_stage == 'deleted':
            shutil.rmtree(run_path)
            print(run_path,"deleted")
        else:
            print(run_path,"active")
    except:
        shutil.rmtree(run_path)
        print(run_path,"deleted")

convert_experiment("/home/work/model/multi_input_model/mlruns/56")